In [87]:
import pandas as pd
df = pd.read_csv('Data/arc.csv')
print(df.head())

          name  version                                             name.1  \
0  ArcPlatform        1     pl.arcilook.controller.JabberAccountController   
1  ArcPlatform        1  pl.arcilook.controller.settings.TaskController...   
2  ArcPlatform        1        pl.arcilook.exception.ServiceLogicException   
3  ArcPlatform        1                 pl.arcilook.service.ContactService   
4  ArcPlatform        1   pl.arcilook.exception.NotProjectManagerException   

   wmc  dit  noc  cbo  rfc  lcom  ca  ...       dam  moa       mfa       cam  \
0    7    1    0    6   19     7   0  ...  1.000000    2  0.000000  0.366667   
1   13    1    0    1   17    64   0  ...  1.000000    0  0.000000  0.461538   
2    7    4    4   48   12    19  47  ...  0.333333    0  0.866667  0.458333   
3    4    1    0    4    4     6   2  ...  0.000000    0  0.000000  1.000000   
4    4    5    0    2    8     6   1  ...  0.000000    0  1.000000  0.666667   

   ic  cbm        amc  max_cc  avg_cc  bug  
0   0

In [88]:
df = df.drop(['name', 'version', 'name.1'], axis=1)

print(df.head())

   wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  ...       dam  moa  \
0    7    1    0    6   19     7   0   6    6  0.750000  ...  1.000000    2   
1   13    1    0    1   17    64   0   1   12  0.916667  ...  1.000000    0   
2    7    4    4   48   12    19  47   1    6  0.944444  ...  0.333333    0   
3    4    1    0    4    4     6   2   2    4  2.000000  ...  0.000000    0   
4    4    5    0    2    8     6   1   1    4  2.000000  ...  0.000000    0   

        mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0  0.000000  0.366667   0    0  15.142857       3  1.1429    0  
1  0.000000  0.461538   0    0   7.461538       3  1.3846    0  
2  0.866667  0.458333   0    0   4.857143       1  0.2857    0  
3  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
4  1.000000  0.666667   0    0   4.000000       0  0.0000    0  

[5 rows x 21 columns]


In [89]:
import numpy as np

features = np.array(df.drop(['bug'], axis=1), dtype='float32')
labels = np.array(df['bug'], dtype='uint8')

labels[labels>0] = 1

print(labels.shape)

(234,)


In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(features)
features = scaler.transform(features)

In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [92]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train, y_train)
counts = np.bincount(y_train)
print(counts)

[165 165]


In [93]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            5, activation="relu", input_shape=(X_train.shape[-1],)
        ),
        keras.layers.Dense(5, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(5, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 5)                 105       
                                                                 
 dense_29 (Dense)            (None, 5)                 30        
                                                                 
 dropout_14 (Dropout)        (None, 5)                 0         
                                                                 
 dense_30 (Dense)            (None, 5)                 30        
                                                                 
 dropout_15 (Dropout)        (None, 5)                 0         
                                                                 
 dense_31 (Dense)            (None, 1)                 6         
                                                                 
Total params: 171
Trainable params: 171
Non-trainable 

In [94]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)


model.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=5,
    verbose=2,
)

Epoch 1/5
330/330 - 3s - loss: 0.7096 - fn: 45.0000 - fp: 108.0000 - tn: 57.0000 - tp: 120.0000 - precision: 0.5263 - recall: 0.7273 - 3s/epoch - 8ms/step
Epoch 2/5
330/330 - 1s - loss: 0.6871 - fn: 18.0000 - fp: 130.0000 - tn: 35.0000 - tp: 147.0000 - precision: 0.5307 - recall: 0.8909 - 1s/epoch - 3ms/step
Epoch 3/5
330/330 - 1s - loss: 0.6768 - fn: 18.0000 - fp: 129.0000 - tn: 36.0000 - tp: 147.0000 - precision: 0.5326 - recall: 0.8909 - 1s/epoch - 4ms/step
Epoch 4/5
330/330 - 1s - loss: 0.6794 - fn: 7.0000 - fp: 134.0000 - tn: 31.0000 - tp: 158.0000 - precision: 0.5411 - recall: 0.9576 - 1s/epoch - 4ms/step
Epoch 5/5
330/330 - 1s - loss: 0.6712 - fn: 13.0000 - fp: 121.0000 - tn: 44.0000 - tp: 152.0000 - precision: 0.5568 - recall: 0.9212 - 1s/epoch - 4ms/step


In [95]:
scores = model.evaluate(X_test, y_test) 

2/2 [==============================] - 1s 5ms/step - loss: 0.6528 - fn: 0.0000e+00 - fp: 26.0000 - tn: 16.0000 - tp: 5.0000 - precision: 0.1613 - recall: 1.0000


In [96]:
df2 = pd.read_csv('Data/tomcat.csv')
x_target = np.array(df2.drop(['name', 'version', 'name.1', 'bug'], axis=1), dtype='float32')
y_target = np.array(df2['bug'], dtype='int8')
y_target[y_target>0] = 1

In [97]:
model.evaluate(x_target, y_target)

27/27 [==============================] - 1s 4ms/step - loss: 13.5469 - fn: 54.0000 - fp: 227.0000 - tn: 554.0000 - tp: 23.0000 - precision: 0.0920 - recall: 0.2987    


[13.546855926513672,
 54.0,
 227.0,
 554.0,
 23.0,
 0.09200000017881393,
 0.29870128631591797]

In [98]:
y_target_pred = model.predict(x_target)
y_target_pred = np.round(y_target_pred)

In [99]:
from sklearn.metrics import f1_score, matthews_corrcoef
print(f1_score(y_target, y_target_pred))
print(matthews_corrcoef(y_target, y_target_pred))

0.14067278287461776
0.005062356062541434


In [100]:
from sklearn.metrics import confusion_matrix, recall_score

def GMean_and_Balance(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    recall = recall_score(y_true, y_pred)
    
    pf = FP / (FP + TN)
    
    GMean = np.sqrt(recall * (1-pf))
    
    Balance = 1 - (np.sqrt(pf**2 + (1 - recall)**   2) / np.sqrt(2))
    
    return GMean, Balance

In [101]:
GMean_and_Balance(y_target, y_target_pred)

(0.46030736193618477, 0.4632043964121846)